In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

from pyspark.sql.window import Window


**Q1 While ingesting customer data from an external source, you notice duplicate entries. How would you remove duplicates and retain only the latest entry based on a timestamp column?**

In [0]:
data = [("101", "2023-12-01", 100), ("101", "2023-12-02", 150), 
        ("102", "2023-12-01", 200), ("102", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

### Prodcut_id-  Duplicate entries 
# latest date - timestamp coloumn 


In [0]:

## Casting date col  string to date format  , DateType isApi in pyspark or sql 
df = df.withColumn('date' , col('date').cast(DateType()))

# or we cam doo --- df = df.withColumn('date', to_date(col('date')))

#Drop Duplicates , accending[ 1,0] bcz prodcut id in asscending order and date in desscending order

df = df.orderBy('product_id' , 'date' , ascending = [1,0]).dropDuplicates(subset=['product_id']).display()

**2. While processing data from multiple files with inconsistent schemas, you need to merge them into a single DataFrame. How would you handle this inconsistency in PySpark?**

In [0]:
# mergeSchema

df = spark.read.format('parquet') \
    .option('mergeSchema', True) \
    .load('/FileStore/Data/Datafiles/*.parquet')

# Don't Use When:
# 1. Performance is Critical
#     mergeSchema is SLOW!
#     Spark has to read metadata of ALL files
#     For 1000 files, reads 1000 schemas!

# 2. Schema is Consistent 
#     All files have same schema
#     No need for mergeSchema (default is faster)
df = spark.read.parquet('/data/*.parquet')  # Faster!

# 3. Very Large Number of Files
#     10,000 parquet files
#     mergeSchema will take forever!
#     Better: Define schema explicitly




# Summary
# mergeSchema = True means:

# Read all parquet file schemas
# Merge them into one unified schema
# Handle missing columns automatically
# Fill missing values with null

# When to use:

# Parquet files only
# Schema evolution over time
# Manageable number of files
# Want simple solution

# When NOT to use:

# Many files (slow!)
# Non-parquet formats
# Known schema (define explicitly)
# Performance critical

In [0]:
# inconsistency in Pyspark , while processing data from multiple files with inconsistent schemas



#what to use : 
    # 1. Always Use unionByName 
    # 2.  Define Master Schema
        """Have a reference schema
        All data must conform to it"""
        master_schema = {...}

    # 3.  Add Metadata
        # Track data lineage
        df = df.withColumn("source_file", lit("jan_sales.csv")) \
       .withColumn("load_date", current_timestamp())

    # 4. Validate Early
        # Check schema before processing
        required_cols = ["id", "product", "price"]
        if not all(col in df.columns for col in required_cols):
            raise ValueError("Invalid schema!")
 
    # 5. Log Schema Changes
        # Log what you're doing
        print(f"Original columns: {df1.columns}")
        print(f"Adding columns: {missing_cols}")

    ##  Key Takeaway: Use unionByName(allowMissingColumns=True) - it handles 90% of cases automatically!    print(f"Final columns: {result.columns}")






## Q : what are the key diffrence between spark and haddop mapreduce , interms of flexiblity scaliblity : 

## Ans : 

# Spark vs Hadoop MapReduce - Key Differences

## 1. Speed & Performance

**Spark:**
- In-memory processing
- 100x faster for iterative algorithms
- 10x faster for disk-based operations

**MapReduce:**
- Disk-based processing
- Reads/writes to disk after each operation
- Slower for iterative jobs

---

## 2. Ease of Use

**Spark:**
- High-level APIs (Python, Scala, Java, R, SQL)
- Rich libraries (SQL, ML, Streaming, GraphX)
- Interactive shell available
- Less code (2-5x less than MapReduce)

**MapReduce:**
- Low-level API (only Java primarily)
- Verbose code
- No interactive mode
- Limited built-in libraries

---

## 3. Processing Model

**Spark:**
- Batch + Real-time streaming
- Iterative processing (ML algorithms)
- Interactive queries
- DAG execution engine

**MapReduce:**
- Only batch processing
- Two-step: Map → Reduce
- Not suitable for iterative jobs
- No interactive mode

---

## 4. Flexibility

**Spark:**
- Multiple workloads: SQL, streaming, ML, graphs
- Unified engine for everything
- Can read from multiple sources

**MapReduce:**
- Only batch processing
- Separate tools needed for streaming, SQL
- Limited to Hadoop ecosystem

---

## 5. Scalability

**Spark:**
- Scales horizontally
- Better resource utilization
- Dynamic resource allocation
- Works on YARN, Mesos, K8s, Standalone

**MapReduce:**
- Scales horizontally
- Only on YARN
- Static resource allocation

---

## 6. Fault Tolerance

**Spark:**
- RDD lineage (recompute lost partitions)
- In-memory recovery
- Faster recovery

**MapReduce:**
- Replication (data copied 3x)
- Disk-based recovery
- Slower recovery

---

## 7. Real-Time Processing

**Spark:**
- ✅ Yes (Spark Streaming, Structured Streaming)
- Micro-batch processing
- Near real-time

**MapReduce:**
- ❌ No
- Only batch processing
- Not suitable for real-time

---

## Quick Comparison Table

| Feature | Spark | MapReduce |
|---------|-------|-----------|
| Speed | 100x faster | Baseline |
| Memory | In-memory | Disk-based |
| Ease of Use | Easy (Python, SQL) | Hard (Java) |
| Real-time | Yes | No |
| Iterative | Excellent | Poor |
| Fault Tolerance | Lineage | Replication |
| APIs | High-level | Low-level |

---

## When to Use What?

**Use Spark:**
- Machine Learning
- Real-time analytics
- Interactive queries
- Iterative algorithms
- Complex workflows

**Use MapReduce:**
- Simple batch jobs
- Already invested in MapReduce
- Stable, proven workloads
- One-time transformations

---

## Interview Answer (Short):

"Spark is 100x faster than MapReduce because it processes data in-memory instead of disk. Spark supports real-time streaming, machine learning, and SQL with easy-to-use APIs in Python, Scala, and Java. MapReduce is disk-based, only does batch processing, and requires verbose Java code. Spark uses RDD lineage for fault tolerance while MapReduce uses data replication. For modern big data workloads, Spark is the clear choice due to its speed, flexibility, and unified platform for batch, streaming, ML, and SQL."

---

## Key Points to Remember:

1. **Speed**: Spark in-memory vs MapReduce disk
2. **Flexibility**: Spark does everything vs MapReduce only batch
3. **Ease**: Spark Python/SQL vs MapReduce Java
4. **Real-time**: Spark yes vs MapReduce no
5. **Recovery**: Spark lineage vs MapReduce replication

**4. You are working with a real-time data pipeline, and you notice missing values in your streaming data Column - Category. How would you handle null or missing values in such a scenario?**

**df_stream = spark.readStream.schema("id INT, value STRING").csv("path/to/stream")**

In [0]:
## Handling missing vlaue in your streaming data when working with columns - category , 
## HJandle Null or missign value in your streaming data
df_stream = spark.readStream.schema("id INT, value STRING").csv("path/to/stream")
df_stream_cleaned = df_stream.fillna({"Category": "N/A"})


**5. You need to calculate the total number of actions performed by users in a system. How would you calculate the top 5 most active users based on this information?**

In [0]:
data = [("user1", 5), ("user2", 8), ("user3", 2), ("user4", 10), ("user2", 3)]
columns = ["user_id", "actions"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = (df.groupBy("user_id")\
    .agg(sum("actions").alias("total_actions"))\
        .orderBy("total_actions", ascending= False)\
            .limit(5))



display(df)
                                                                                            

**6. While processing sales transaction data, you need to identify the most recent transaction for each customer. How would you approach this task?**

In [0]:
data = [("cust1", "2023-12-01", 100), ("cust2", "2023-12-02", 150),
        ("cust1", "2023-12-03", 200), ("cust2", "2023-12-04", 250)]
columns = ["customer_id", "transaction_date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df = df.withColumn('transaction_date', col('transaction_date').cast(DateType()))

df = df.withColumn('flag' , dense_rank().over(Window.partitionBy('customer_id').orderBy(col('transaction_date').desc()))).filter(col('flag') == 1)

df.display()

**7. You need to identify customers who haven’t made any purchases in the last 30 days. How would you filter such customers?**

In [0]:
data = [("cust1", "2025-12-01"), ("cust2", "2024-11-20"), ("cust3", "2024-11-25")]
columns = ["customer_id", "last_purchase_date"]

df = spark.createDataFrame(data, columns)

df.display()

In [0]:
# current_date() → today’s date (system date)

# datediff(A, B) → number of days between A and B

df = df.withColumn('last_purchase_date', to_date('last_purchase_date'))
df = df.withColumn('gap' , datediff(current_date(), 'last_purchase_date')).filter(col('gap')> 30) 

df.display()

**8. While analyzing customer reviews, you need to identify the most frequently used words in the feedback. How would you implement this?**

In [0]:
data = [("customer1", "The product is great"), ("customer2", "Great product, fast delivery"), ("customer3", "Not bad, could be better")]
columns = ["customer_id", "feedback"]

df = spark.createDataFrame(data, columns)

df.display()

In [0]:
# Text into array using split fun 
# using explode , split 
df = df.withColumn('feedback' , lower('feedback')).withColumn('feedback' , explode(split('feedback',' ')))
df_grp = df.groupBy("feedback").agg(count("feedback").alias('wordcount'))
df_grp.display()

**9. You need to calculate the cumulative sum of sales over time for each product. How would you approach this?**

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-02", 200),
        ("product1", "2023-12-03", 150), ("product2", "2023-12-04", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
#total sum , of sales , we'll using ag
# cast date that is in string format into date format 

df = df.withColumn('date', to_date('date'))
df = df.withColumn('CumSum' , sum('sales').over(Window.partitionBy('product_id').orderBy('date')) )

df.display()

**10. While preparing a data pipeline, you notice some duplicate rows in a dataset. How would you remove the duplicates without affecting the original order?**

In [0]:
data = [("John", 25), ("Jane", 30), ("John", 25), ("Alice", 22)]
columns = ["name", "age"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
# in this and previous one we apply row level transformation 

df = df.withColumn('rowflag', row_number().over(Window.partitionBy('name').orderBy('age'))).filter(col('rowflag') == 1)

df.display()

**11. You are working with user activity data and need to calculate the average session duration per user. How would you implement this?**

In [0]:
data = [("user1", "2023-12-01", 50), ("user1", "2023-12-02", 60), 
        ("user2", "2023-12-01", 45), ("user2", "2023-12-03", 75)]
columns = ["user_id", "session_date", "duration"]
df = spark.createDataFrame(data, columns)

df.display()

In [0]:
df = df.groupBy('user_id').agg(avg("duration").alias('avg_duration'))
df.display()

**12. While analyzing sales data, you need to find the product with the highest sales for each month. How would you accomplish this?**

In [0]:
data = [("product1", "2023-12-01", 100), ("product2", "2023-12-01", 150), 
        ("product1", "2023-12-02", 200), ("product2", "2023-12-02", 250)]
columns = ["product_id", "date", "sales"]
df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = df.withColumn('date', to_date('date'))

df = df.withColumn('date' , month('date')).groupBy("date" , "product_id").agg(sum("sales").alias('sales_sum'))
df = df.withColumn('ranking' , dense_rank()\
    .over(Window.partitionBy('date')\
    .orderBy(col('sales_sum').desc())))\
    .filter(col('ranking') == 1)

df.display()


**what is role of SparkContext in pyspark** 


**13. You are working with a large Delta table that is frequently updated by multiple users. The data is stored in partitions, and sometimes updates can cause inconsistent reads due to concurrent transactions. How would you ensure ACID compliance and avoid data corruption in PySpark?**

**14. You need to process a large dataset stored in PARQUET format and ensure that all columns have the right schema (Almost). How would you do this?**


In [0]:
df = spark.read.format('parquet')\
    .option('inferSchema', True)\
        .load('path')


**15. You are reading a CSV file and need to handle corrupt records gracefully by skipping them. How would you configure this in PySpark?**

In [0]:
df = spark.read.format('csv')\
    .option('mode', "DROPMALFORMED")\
        .load('stagging location')
        

**Q16**
    
**A diffrence betwwen , RDDs , Dataframe , Dataset**

**B What is Query Optimization , catalyst optimizer , join which one**

**C Tell us about spark session**

**D diff btw narrow and wide transformation**

**E what is use of colease and repartition**

**F when to use catch() and Persist()  whats diffrence**

**G whats importance of partition in Pyspark**






# Query Optimization in Spark

## What is Query Optimization?

Query optimization is the process where Spark transforms your query into the most efficient execution plan.

---

## The Flow
```text
User Query (SQL/DataFrame)
        |
        v
    [Parser]
        |
        v
Unresolved Logical Plan
        |
        v
    [Analyzer]
        |
        v
Resolved Logical Plan
        |
        v
    [Optimizer]
        |
        v
Optimized Logical Plan
        |
        v
[Physical Planner]
        |
        v
Multiple Physical Plans
        |
        v
   [Cost Model]
        |
        v
Best Physical Plan
        |
        v
   [Execution]
        |
        v
    Results
```

---

## Two Main Components

### 1. Logical Plan (What to do)
- Describes WHAT operations to perform
- Does not specify HOW to execute
- Abstract representation of query
- Multiple optimization rules applied

**Example:**
```text
Filter (age > 30)
    |
Project (name, age)
    |
Scan (table)
```

---

### 2. Physical Plan (How to do it)
- Describes HOW to execute operations
- Specific execution strategy
- Considers resources and data distribution
- Multiple physical plans generated

**Example:**
```text
HashAggregate
    |
Exchange (shuffle)
    |
FileScan (with pushed filters)
```

---

## Key Difference

| Aspect | Logical Plan | Physical Plan |
|--------|--------------|---------------|
| Focus | What to do | How to do it |
| Level | Abstract | Concrete |
| Examples | Filter, Join, Group | HashJoin, BroadcastJoin, SortMergeJoin |
| Execution | No | Yes |

---

## Example

**Query:**
```python
df.filter(col("age") > 30).select("name", "age")
```

**Logical Plan:**
```text
What: Filter by age, then select columns
```

**Physical Plan:**
```text
How: 
- FileScan with pushed filter (age > 30)
- Project columns (name, age)
- Use 8 partitions
- BroadcastHashJoin if needed
```

---

## Key Point

Query optimization converts your high-level query (Logical Plan) into the fastest execution strategy (Physical Plan) automatically!

**22. You have a dataset containing the names of employees and their departments. You need to find the department with the most employees.**

In [0]:
data = [("Alice", "HR"), ("Bob", "Finance"), ("Charlie", "HR"), ("David", "Engineering"), ("Eve", "Finance")]
columns = ["employee_name", "department"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = df.groupBy("department").agg(count("*").alias("total_employee")).sort('total_employee' , ascending=False)
df.display()

**23. While processing sales data, you need to classify each transaction as either 'High' or 'Low' based on its amount. How would you achieve this using a when condition**

In [0]:
data = [("product1", 100), ("product2", 300), ("product3", 50)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = df.withColumn('price_cat', when(col('sales') > 50 , "High").otherwise("Low"))
df.display()

**24. While analyzing a large dataset, you need to create a new column that holds a timestamp of when the record was processed. How would you implement this and what can be the best USE CASE?**


In [0]:

data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()

In [0]:
df = df.withColumn("processed_time" , current_timestamp())
df.display()

**25. You need to register this PySpark DataFrame as a temporary SQL object and run a query on it. How would you achieve this?**


### if someoene dont want to use python queri , thne change its view , bez they wanna use sql 

In [0]:
data = [("product1", 100), ("product2", 200), ("product3", 300)]
columns = ["product_id", "sales"]

df = spark.createDataFrame(data, columns)
df.display()


In [0]:
df.createOrReplaceTempView("temsqldf")

In [0]:
spark.sql("SELECT * FROM temsqldf")
  